# Exercise 6: Custom CNN with VGG16 Architecture
## Student: JOSEPHMARI CORDERO
## Dataset: Cats vs Dogs Classification

This notebook implements a CNN using VGG16 transfer learning architecture.

**Dataset**: Cats vs Dogs (Custom dataset - 25 points)
**Architecture**: VGG16 with Transfer Learning (50 points)

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [ ]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# DATASET DIRECTORY CONFIGURATION
# Using Cats vs Dogs dataset
# You can download from: https://www.kaggle.com/datasets/salader/dogs-vs-cats
# Or use your own custom dataset with similar structure

train_dir = "cats_dogs_dataset/train"  # Update with your dataset path
test_dir = "cats_dogs_dataset/test"    # Update with your dataset path

print("Dataset directories configured:")
print(f"Training: {train_dir}")
print(f"Testing: {test_dir}")

In [ ]:
# IMAGE PARAMETERS
# VGG16 requires 224x224 input size
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32
EPOCHS = 20

print(f"Image size: {IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training epochs: {EPOCHS}")

In [ ]:
# DATA PREPROCESSING & AUGMENTATION
# Enhanced augmentation for better generalization

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"\nTraining samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")
print(f"Classes: {train_generator.class_indices}")

In [ ]:
# VGG16 ARCHITECTURE WITH TRANSFER LEARNING

# Load pre-trained VGG16 model without top layers
print("Loading VGG16 base model (pre-trained on ImageNet)...")
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Freeze the base model layers (we'll only train the custom top layers)
base_model.trainable = False

print("\nBase VGG16 Model Summary:")
base_model.summary()

# Build the complete model with custom classification head
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

print("\n" + "="*70)
print("COMPLETE VGG16 MODEL WITH CUSTOM LAYERS")
print("="*70)
model.summary()

# Count parameters
total_params = model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable_params = sum([tf.size(w).numpy() for w in model.non_trainable_weights])

print("\n" + "="*70)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {non_trainable_params:,}")
print("="*70)

## VGG16 Architecture Details

### **What is VGG16?**
VGG16 is a convolutional neural network architecture developed by the Visual Geometry Group (VGG) at Oxford. It was a runner-up in the 2014 ImageNet competition.

### **Architecture Highlights:**
1. **16 Weight Layers**: 13 convolutional layers + 3 fully connected layers
2. **Small Filters**: Uses only 3x3 convolution filters throughout
3. **Deep Network**: Stacks multiple conv layers before pooling
4. **Pre-trained Weights**: Trained on ImageNet (1.4M images, 1000 classes)

### **Transfer Learning Approach:**
1. **Base Model (Frozen)**: VGG16 pre-trained layers extract features
2. **Custom Head (Trainable)**: 
   - Flatten layer
   - Dense(256) + Dropout(0.5) with L2 regularization
   - Dense(128) + Dropout(0.5) with L2 regularization
   - Output Dense(1) with sigmoid activation

### **Benefits:**
- Leverages learned features from ImageNet
- Faster training with smaller datasets
- Better generalization
- Reduced overfitting with dropout and L2 regularization

### **Improvements Applied:**
- **L2 Regularization**: Weight decay = 0.001
- **Dropout Layers**: 50% dropout rate
- **Data Augmentation**: Rotation, shift, zoom, flip, shear

In [ ]:
# COMPILE THE MODEL

# Use Adam optimizer with lower learning rate for transfer learning
optimizer = Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully!")
print(f"Optimizer: Adam (lr=0.0001)")
print(f"Loss: Binary Crossentropy")
print(f"Metrics: Accuracy")

In [ ]:
# SET UP CALLBACKS

# ModelCheckpoint: Save the best model
checkpoint = ModelCheckpoint(
    'exercise_6_custom_Cordero.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# EarlyStopping: Stop training if validation loss doesn't improve
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

callbacks = [checkpoint, early_stopping]

print("Callbacks configured:")
print("✓ ModelCheckpoint: Saves best model based on validation accuracy")
print("✓ EarlyStopping: Stops if validation loss doesn't improve for 5 epochs")

In [ ]:
# TRAIN THE VGG16 MODEL

print("="*70)
print("STARTING TRAINING WITH VGG16 TRANSFER LEARNING")
print("="*70)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70)
print("TRAINING COMPLETE!")
print("="*70)
print(f"Model saved as: exercise_6_custom_Cordero.h5")

In [ ]:
# VISUALIZE TRAINING HISTORY

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s', linewidth=2)
ax1.set_title('VGG16 Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss', marker='o', linewidth=2)
ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s', linewidth=2)
ax2.set_title('VGG16 Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history_VGG16_Cordero.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training history plot saved as: training_history_VGG16_Cordero.png")

In [ ]:
# EVALUATE THE VGG16 MODEL

print("Evaluating model on test data...")
test_loss, test_acc = model.evaluate(test_generator)

print("\n" + "="*70)
print("VGG16 MODEL PERFORMANCE")
print("="*70)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print("="*70)

# Display final training metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print("\nFinal Training Metrics:")
print(f"Training Accuracy: {final_train_acc * 100:.2f}%")
print(f"Validation Accuracy: {final_val_acc * 100:.2f}%")
print(f"Training Loss: {final_train_loss:.4f}")
print(f"Validation Loss: {final_val_loss:.4f}")
print("="*70)

In [ ]:
# LOAD AND TEST SAVED MODEL

print("Loading saved model from disk...")
loaded_model = tf.keras.models.load_model('exercise_6_custom_Cordero.h5')

print("\nTesting loaded model...")
loaded_loss, loaded_acc = loaded_model.evaluate(test_generator, verbose=0)

print("\n" + "="*70)
print("LOADED MODEL VERIFICATION")
print("="*70)
print(f"Loaded Model Test Accuracy: {loaded_acc * 100:.2f}%")
print(f"Loaded Model Test Loss: {loaded_loss:.4f}")
print("="*70)
print("\n✓ Model successfully saved and loaded!")
print(f"✓ File: exercise_6_custom_Cordero.h5")

In [ ]:
# PREDICTION FUNCTION

from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_custom_Cordero.h5'):
    """
    Predict the class of an image using the trained VGG16 model
    Args:
        img_path: Path to the image file
        model_path: Path to the trained model
    Returns:
        label: Predicted class label
        confidence: Prediction confidence (0-1)
    """
    # Load model
    model = tf.keras.models.load_model(model_path)
    
    # Load and preprocess image
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make prediction
    pred = model.predict(img_array, verbose=0)[0,0]
    
    # Determine label based on class indices
    # Adjust these labels based on your actual dataset classes
    label = "Class 1" if pred >= 0.5 else "Class 0"
    confidence = pred if pred >= 0.5 else (1 - pred)
    
    print(f"Image: {img_path}")
    print(f"Prediction: {label}")
    print(f"Confidence: {confidence * 100:.2f}%")
    print(f"Raw score: {pred:.4f}")
    print(f"{'-'*50}\n")
    
    return label, confidence

print("Prediction function defined successfully!")
print("Usage: predict_image('path/to/image.jpg')")

## Project Summary

### Assignment Requirements Met:

#### ✅ **Part 1: Custom Dataset (25 points)**
- **Dataset Used**: Cats vs Dogs
- **Dataset Source**: Custom dataset (not Muffin vs Chihuahua)
- **Classes**: Binary classification (2 classes)
- **Dataset Structure**: Train and test directories with proper organization

#### ✅ **Part 2: CNN Architecture Selection (50 points)**
- **Architecture**: VGG16 with Transfer Learning
- **Why VGG16?**
  - Deep architecture (16 layers)
  - Pre-trained on ImageNet
  - Proven performance on image classification
  - Excellent for transfer learning
  
### Model Specifications:
- **Input Size**: 224x224x3 (RGB images)
- **Base Model**: VGG16 (frozen, pre-trained on ImageNet)
- **Custom Layers**: 
  - Flatten
  - Dense(256) + Dropout(0.5) + L2 regularization
  - Dense(128) + Dropout(0.5) + L2 regularization
  - Dense(1) with sigmoid activation
- **Total Parameters**: ~15M (most frozen)
- **Trainable Parameters**: ~2M (custom layers only)

### Training Configuration:
- **Optimizer**: Adam (lr=0.0001)
- **Loss**: Binary Crossentropy
- **Batch Size**: 32
- **Epochs**: 20 (with early stopping)
- **Data Augmentation**: Rotation, shift, zoom, flip, shear
- **Regularization**: L2 (0.001) + Dropout (0.5)

### Files Generated:
1. **exercise_6_custom_Cordero.h5** - Trained model
2. **training_history_VGG16_Cordero.png** - Training plots

### Student: JOSEPHMARI CORDERO